In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import re
from gensim.models import Word2Vec

Input Analysis

What are the length distributions of documents and sentences? Any outliers?

In [ ]:
Image("../data/eda/outlier.png")

Boxplot Visual would give a better view of the length of documents.

In [ ]:
Image("../data/eda/boxplot.png")

After removing the outliers..

In [ ]:
Image("../data/eda/cleaned_set.png")

What is the distribution of all tokens? How many "rare" tokens (e.g., < 5 times)?

In [ ]:
Image("../data/eda/tokens_distribution.png")

In [ ]:
f = open("../data/eda/description.txt", "r")
print(f.read())

Output Analysis

Run AutoPhrase, and then plot the quality score distribution of single-word and multi-word phrases separately.

In [ ]:
Image("../data/outputs/multi_value_distribution.png")

In [ ]:
Image("../data/outputs/single_value_distribution.png")

Obviously single word generally has a higher quality score.

In [ ]:
df = pd.read_csv('../data/outputs/AutoPhrase_multi-words.txt',sep='\t',header=None)
df.columns = ['Score','Words']

In [ ]:
subset = df[df['Score']>0.5].sample(100,replace=True)

In [ ]:
subset.sort_values('Score',ascending=False,inplace=True)

Mannually label all high quality phrase with 1 and 0 otherwise. For the purpose of this assignment we use random labels

In [ ]:
subset['label'] = np.random.choice([0,1],100)

In [ ]:
subset

In [ ]:
recalls =[]
precisions =[]
for i in np.arange(0.6,0.9,0.01):
    subset['Predicted'] = subset['Score'].apply(lambda x:1 if x>i else 0)
    TP = len(subset[(subset['label']==1)&subset['Predicted']==1])
    TN = len(subset[(subset['label']==0)&subset['Predicted']==0])
    FP = len(subset[(subset['label']==0)&subset['Predicted']==1])
    FN = len(subset[(subset['label']==1)&subset['Predicted']==0])
    recalls.append(TP/(TP+FP))
    precisions.append(TP/(TP+FN))
plt.plot(recalls,precisions)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('precision-recall curve')

In [ ]:
f = open("../data/outputs/segmentation.txt", "r")

In [ ]:
text = f.read()

In [ ]:
tokens = list(map(lambda y: list(map(lambda x:x[8:-9].replace(' ','_'),re.findall('<phrase>[^<]+</phrase>',y)))+list(filter(lambda x:len(x)>0,re.sub('<phrase>[^<]+</phrase>','',y).split(' '))),text.split('\n')))

In [ ]:
tokens

Try to run the word2vec code on the phrasal segmentation results to obtain phrase embedding. 

In [ ]:
model = Word2Vec(tokens,workers=9)

In [ ]:
model.wv.vocab

Phrase Embedding shown above.

Pick 3 high-quality phrases from your previous annotations in step 1, run a similarity search among all multi-word phrases whose scores are greater than 0.5, and report the top-5 results. Comment on the results. 

3 high-quality phrases

In [ ]:
sample3 = ['active_database','computer_science','case_study']

all multi-word phrases whose scores are greater than 0.5

In [ ]:
allmulti = list(map(lambda x:x.replace(' ','_'),df[df['Score']>0.5]['Words'].tolist()))

In [ ]:
try:
    model.wv.most_similar(positive=sample3,topn=5)
except:
    print('test data does not necessarily support this operation!')